In [2]:
%load_ext autoreload
%autoreload 2

In [1]:
import sys
#project_root = os.path.abspath(os.path.join(os.getcwd(), ".."))
#sys.path.insert(0, project_root)
sys.path.append("..") 
sys.path.append("../src")

In [3]:
from src.checkpoint_storage import save_all_checkpoints_v2,load_all_checkpoints_v2

In [4]:
checkpoint=load_all_checkpoints_v2("har_pre_model_20250801_144835")

In [6]:
checkpoint










{}